In [0]:
#Connect to ADLS
# Connection String Variables
adlsAccountName = "adlsimdbproject"
adlsContainerName = "exportedfiles-imdbproject"
adlsFolderName = "ExportedFiles"
mountPoint = "/mnt/csvFiles"

# Client ID - from Azure Key Vault using Secret Scope
applicationId = dbutils.secrets.get(scope="imdbprojectKeyVault",key="ClientId")

# Application (Client) - - from Azure Key Vault using Secret Scope
authenticationKey = dbutils.secrets.get(scope="imdbprojectKeyVault",key="ClientSecret")

# Directory (Tenant) ID - - from Azure Key Vault using Secret Scope
tenandId = dbutils.secrets.get(scope="imdbprojectKeyVault",key="TenatId")

# Dynamic End Point to ADLS-u
endpoint = "https://login.microsoftonline.com/" + tenandId + "/oauth2/token"
source = "abfss://" + adlsContainerName + "@" + adlsAccountName + ".dfs.core.windows.net/" + adlsFolderName

# Connect using Service Principal secrets and OAuth
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": applicationId,
           "fs.azure.account.oauth2.client.secret": authenticationKey,
           "fs.azure.account.oauth2.client.endpoint": endpoint}

# Mounting ADLS Storage => DBFS
 
if not any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
  dbutils.fs.mount(
    source = source,
    mount_point = mountPoint,
    extra_configs = configs)

In [0]:
# reference import

from pyspark.sql import SQLContext
from pyspark.sql.functions import explode
from pyspark.sql.types import *
from pyspark.sql.functions import col, split, concat_ws
from pyspark.sql import functions as sf
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [0]:
# Create dataframes from 2 tsv files

dfBasics = spark.read.csv("/mnt/csvFiles/Basics.csv", sep=r'\t', header=True).select('tconst','titleType','primaryTitle','originalTitle','isAdult','startYear','endYear','runtimeMinutes','genres')
dfRatings = spark.read.csv("/mnt/csvFiles/Ratings.csv", sep=r'\t', header=True).select('tconst','averageRating','numVotes')

df = (dfBasics.alias('dfBasics').join(dfRatings.alias('dfRatings'),
                               on = dfBasics['tconst'] == dfRatings['tconst'],
                               how = 'inner')
                         .select('dfBasics.*',
                                 'dfRatings.averageRating')
)


# start transform
df2 = df.withColumn("genres", split(col("genres"), ",").cast("array<string>"))

# explosion in dataset - transform comma separated velues in columns in rows
df3 = df2.withColumn("originalGenres",explode(df2.genres))
df4 = df3.withColumn('webLocation', sf.concat(sf.lit('https://www.imdb.com/title'),sf.lit('/'), sf.col('tconst')))
df4.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------------+--------------------+
 tconst|titleType| primaryTitle| originalTitle|isAdult|startYear|endYear|runtimeMinutes| genres|averageRating|originalGenres| webLocation|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------------+--------------------+
tt0000015| short| Autour d'une cabine| Autour d'une cabine| 0| 1894| \N| 2| [Animation, Short]| 6.1| Animation|https://www.imdb....|
tt0000015| short| Autour d'une cabine| Autour d'une cabine| 0| 1894| \N| 2| [Animation, Short]| 6.1| Short|https://www.imdb....|
tt0000019| short| The Clown Barber| The Clown Barber| 0| 1898| \N| \N| [Comedy, Short]| 5.3| Comedy|https://www.imdb....|
tt0000019| short| The Clown Barber| The Clown Barber| 0| 1898| \N| \N| [Comedy, Short]| 5.3| Short|https://www.imdb....|
tt0000058| short|Cortège de tzar a...|Cortège de tzar a...| 0| 1896| \N| \N|[Documentary, Short]| 4.3| Documentary|https://www.imdb....|
tt0000058| short|Cortège de tzar a...|Cortège de tzar a...| 0| 1896| \N| \N|[Documentary, Short]| 4.3| Short|https://www.imdb....|
tt0000095| short|The Mysterious Paper| Le papier protée| 0| 1896| \N| \N| [Short]| 4.8| Short|https://www.imdb....|
tt0000108| short|Rip Leaving Sleep...|Rip Leaving Sleep...| 0| 1896| \N| 1| [Drama, Short]| 4.4| Drama|https://www.imdb....|
tt0000108| short|Rip Leaving Sleep...|Rip Leaving Sleep...| 0| 1896| \N| 1| [Drama, Short]| 4.4| Short|https://www.imdb....|
tt0000145| short| En classe| En classe| 0| 1897| \N| \N| [Comedy, Short]| 3.8| Comedy|https://www.imdb....|
tt0000145| short| En classe| En classe| 0| 1897| \N| \N| [Comedy, Short]| 3.8| Short|https://www.imdb....|
tt0000146| short|Le cocher de fiac...|Le cocher de fiac...| 0| 1897| \N| \N| [Comedy, Short]| 5.2| Comedy|https://www.imdb....|
tt0000146| short|Le cocher de fiac...|Le cocher de fiac...| 0| 1897| \N| \N| [Comedy, Short]| 5.2| Short|https://www.imdb....|
tt0000174| short|Výstavní párkar a...|Výstavní párkar a...| 0| 1898| \N| 1| [Comedy, Short]| 4.7| Comedy|https://www.imdb....|
tt0000174| short|Výstavní párkar a...|Výstavní párkar a...| 0| 1898| \N| 1| [Comedy, Short]| 4.7| Short|https://www.imdb....|
tt0000186| short| Le cuirassé Maine| Le cuirassé Maine| 0| 1898| \N| \N| [Short, War]| 4.2| Short|https://www.imdb....|
tt0000186| short| Le cuirassé Maine| Le cuirassé Maine| 0| 1898| \N| \N| [Short, War]| 4.2| War|https://www.imdb....|
tt0000188| short| The Deserter| The Deserter| 0| 1898| \N| \N| [Drama, Short]| 4.0| Drama|https://www.imdb....|
tt0000188| short| The Deserter| The Deserter| 0| 1898| \N| \N| [Drama, Short]| 4.0| Short|https://www.imdb....|
tt0000264| short|Summoning the Spi...| Évocation spirite| 0| 1899| \N| 1|[Fantasy, Horror,...| 5.7| Fantasy|https://www.imdb....|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------------+--------------------+
only showing top 20 rows

In [0]:
#Load Back to Azure SQL Database


userName = dbutils.secrets.get(scope="imdbprojectKeyVault",key="userName")
passWord = dbutils.secrets.get(scope="imdbprojectKeyVault",key="passWord")
jdbcUrl = dbutils.secrets.get(scope="imdbprojectKeyVault",key="jdbcUrl")

# "jdbc:sqlserver://imdbdatabaseserver.database.windows.net:1433;database=IMdbDataBase"

# return array in column data type 
df4.withColumn('genres', concat_ws(',', 'genres')).write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", jdbcUrl ) \
    .option("dbtable", "dbo.RatedMovies") \
    .option("user", userName) \
    .option("password", passWord) \
    .save()





In [0]:
# Unmount 
if any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
  dbutils.fs.unmount(mountPoint)

/mnt/csvFiles has been unmounted.